In [1]:
import torch
import sys
from torch import nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torch.nn.functional as F
from torchvision import datasets, transforms,models
from torchvision.models import resnet50, ResNet50_Weights, vgg11, vgg16,\
    alexnet, VGG11_Weights, VGG16_Weights, AlexNet_Weights
import time
import os
import copy
from torch.utils.data import sampler
#from utils import progress_bar

/home/ubuntu/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
path=os.getcwd()

In [3]:
import logging
# from EDGE_4_4_1 import EDGE
# from matplotlib.ticker import MaxNLocator
# plt.style.use('ggplot')
log = logging.getLogger("sampleLogger")

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
class Network():
  def __init__(self, device, arch, pretrained=False):
        "docstring"
        self.preprocess = None
        self.model = None
        self.arch = arch
        self.pretrained = pretrained
        self.device = device
  def set_model(self):
        if self.arch == "vgg11":
            if self.pretrained:
                weights = VGG11_Weights.IMAGENET1K_V1
                self.preprocess = weights.transforms()
                self.model = vgg11(weights=weights).to(self.device)
                self.model.eval()
            else:
                self.model = vgg11().to(self.device)
        elif self.arch == "vgg16":
            if self.pretrained:
                weights = VGG16_Weights.IMAGENET1K_V1
                self.preprocess = weights.transforms()
                self.model = vgg16(weights=weights).to(self.device)
                self.model.eval()
            else:
                self.model = vgg16().to(self.device)
        elif self.arch == "resnet":
            if self.pretrained:
                weights = ResNet50_Weights.DEFAULT
                self.preprocess = weights.transforms()
                self.model = resnet50(weights=weights).to(self.device)
                self.model.eval()
            else:
                self.model = resnet50().to(self.device)
        elif self.arch == "alexnet":
            if self.pretrained:
                weights = AlexNet_Weights.IMAGENET1K_V1
                self.preprocess = weights.transforms()
                self.model = alexnet(weights=weights).to(self.device)
                self.model.eval()
            else:
                self.model = alexnet().to(self.device)
        else:
            sys.exit("Wrong architecture")
        return self.model
  def trained_enough(self, accuracy, dataloader, loss_fn, optimizer, epochs, device):   # I think it is not used in this code.
        i = 0
        while accuracy < .20:
            accuracy, _ = train(self.model, dataloader, loss_fn, optimizer,
                                epochs, device)
            log.debug(f"{i} epoch extra training, accuracy: {100 * accuracy}")
            i += 1

In [6]:
batch_size=128

network = Network(device,"resnet", True)
model= network.set_model()
model.conv1= nn.Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
model.maxpool = nn.Identity()
model.fc=nn.Linear(2048, 10)
model=model.to(device)
batch_size=128

for name,module in model.named_modules():
    print(name,module,'\n_______________\n')

# Loading the Data

In [7]:
# Data
print('==> Preparing data..')
transform_train = transforms.Compose([
    transforms.RandomResizedCrop(224),  #first crop the image randomly and then resize it.
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
#(0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)
])

transform_test = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=128, shuffle=True, num_workers=2)

testset = datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=100, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')


==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified


# Fine tunning model for renet 50 on Cifar 10.

best_acc = 0  # best test accuracy
network = Network(device,"resnet", False)
model= network.set_model()
#model.conv1= nn.Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
#model.maxpool = nn.Identity()
model.fc=nn.Linear(2048, 10)
model=model.to(device)
#criterion = nn.CrossEntropyLoss()
#optimizer = optim.SGD(model.parameters(), lr=0.1,
                     # momentum=0.9, weight_decay=5e-4)
#scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=40)

In [8]:
Average_train_accuracy=0
Average_test_accuracy=0

for iter in range(5):
    epoch_number=50
    best_acc = 0  # best test accurac 
    network = Network(device,"resnet", True)
    model= network.set_model()
    #model.conv1= nn.Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    #model.maxpool = nn.Identity()
    model.fc=nn.Linear(2048, 10)
    model=model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.1,
                          momentum=0.9)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epoch_number)
    
    train_test_accuracy_epochs=pd.DataFrame([[0 for i in range(epoch_number)],[0 for i in range(epoch_number)]],columns=[i for i in range(epoch_number)])
   
    for epoch in range(0, epoch_number):
        
        #training
        print('\nEpoch: %d' % epoch)
        model.train()
        train_loss = 0
        correct = 0
        total = 0
        for batch_idx, (inputs, targets) in enumerate(trainloader):
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
            scheduler.step()
        
#         print(epoch, 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
#                          % (train_loss/len(trainloader), 100.*correct/total, correct, total))
        #saving epoch train in this iter in a dataframe to plot later.
        train_test_accuracy_epochs.iloc[0,epoch]=100.*correct/total
        
        #testing 
        model.eval()
        test_loss = 0
        test_correct = 0
        test_total = 0
        with torch.no_grad():
            for batch_idx, (inputs, targets) in enumerate(testloader):
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, targets)

                test_loss += loss.item()
                _, predicted = outputs.max(1)
                test_total += targets.size(0)
                test_correct += predicted.eq(targets).sum().item()

                # print(batch_idx, len(testloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                #              % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))
        # print(epoch, 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
        #                      % (test_loss/len(testloader), 100.*test_correct/test_total, test_correct, test_total))
        
        train_test_accuracy_epochs.iloc[1,epoch]=100.*test_correct/test_total
        
    Average_train_accuracy+=100.*correct/total
    Average_test_accuracy+=100.*test_correct/test_total
    
    print('{}_Average train Accuracy'.format(iter),Average_train_accuracy)
    print('{}_Average test Accuracy'.format(iter),Average_test_accuracy)
    
    torch.save(model.state_dict(), path+'Weights/resnet_CIFAR_dict-iter{}.pth'.format(iter))
    train_test_accuracy_epochs.to_csv(path+'Accuracies/resnet_CIFAR_epoch accs-iter{}.csv'.format(iter))
    
    print(epoch, 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                             % (test_loss/len(testloader), 100.*test_correct/test_total, test_correct, test_total))
    
    print(epoch, 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                         % (train_loss/len(trainloader), 100.*correct/total, correct, total))
    
Ave_acc_train_test=pd.DataFrame([[Average_train_accuracy/5,Average_test_accuracy/5]],columns=['Avg_Train','Avg_Test'])
Ave_acc_train_test.to_csv(path+'Accuracies/Avg_acc_resnet_CIFAR.csv')
    
    
        


Epoch: 0

Epoch: 1

Epoch: 2

Epoch: 3


KeyboardInterrupt: 